opis dodati

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)


from yellowbrick.target import FeatureCorrelation
from scipy.stats import norm
from scipy import stats


import warnings
warnings.filterwarnings("ignore")
color = sns.color_palette()
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

import os


ModuleNotFoundError: No module named 'pandas'

In [ ]:
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Dataset and attributes exploration

Primary:  
  
id (Id of track generated by Spotify)  
Numerical:  
  
acousticness (Ranges from 0 to 1)  
danceability (Ranges from 0 to 1)  
energy (Ranges from 0 to 1)  
duration_ms (Integer typically ranging from 200k to 300k)  
instrumentalness (Ranges from 0 to 1)  
valence (Ranges from 0 to 1)  
popularity (Ranges from 0 to 100)  
tempo (Float typically ranging from 50 to 150)  
liveness (Ranges from 0 to 1)  
loudness (Float typically ranging from -60 to 0)  
speechiness (Ranges from 0 to 1)  
year (Ranges from 1921 to 2020)  
  
Dummy:  
  
mode (0 = Minor, 1 = Major)  
explicit (0 = No explicit content, 1 = Explicit content)  
  
Categorical:  

key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)  
artists (List of artists mentioned)  
release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)  
name (Name of the song)  

Acousticness: Numerical, Confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.  
Danceability: Numerical, danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.  
Duration_ms: Numerical, the duration of the track in milliseconds.  
Duration_min: Numerical, the duration of the track in minutes.  
Energy: Numerical, Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity.   Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.  
Explicit: Categorical, whether or not the track has explicit lyrics ( true = yes it does; false = no it does not OR unknown).
Id: The Spotify ID for the track.  
Instrumentalness: Numerical, predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.  
Key: Numerical, the estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C#/Db, 2 = D, and so on. If no key was detected, the value is -1.  
Liveness: Numerical, detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.  
Loudness: Numerical, Overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.  
Mode: Numerical, mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.  
Popularity: Numerical, the popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are.  
Release_date: Date the song was released  
Speechiness: Numerical, speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.  
Tempo: Numerical, Overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.  
Valence: Numerical, Measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).  
Year: Year the song was released  

In [ ]:
df = pd.read_csv('data/data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['id'], axis = 1, inplace = True)
df.head()

Explicit vs Non-explicit

In [ ]:
sns.countplot(df['explicit'])

Valence ("pozitivnost" pesme) vs Year


In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['valence'].median().plot()
plt.xlabel('year')
plt.ylabel('Median valence')

Energy (mera intenziteta i aktivnosti) vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['energy'].median().plot()
plt.xlabel('year')
plt.ylabel('Median energy')

Popularity vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['popularity'].median().plot()
plt.xlabel('year')
plt.ylabel('Median popularity')

Loudness vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['loudness'].median().plot()
plt.xlabel('year')
plt.ylabel('Median loudness')

Speechiness vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['speechiness'].median().plot()
plt.xlabel('year')
plt.ylabel('Median speechiness')

Tempo vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['tempo'].median().plot()
plt.xlabel('year')
plt.ylabel('Median tempo')

Liveness vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['liveness'].median().plot()
plt.xlabel('year')
plt.ylabel('Median liveness')

Instrumentalness vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['instrumentalness'].median().plot()
plt.xlabel('year')
plt.ylabel('Median instrumentalness')


Duration of song vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['duration_ms'].median().plot()
plt.xlabel('year')
plt.ylabel('Median duration_ms')

Acousticness vs Year

In [ ]:
plt.figure(figsize = (10,10))
df.groupby('year')['acousticness'].median().plot()
plt.xlabel('year')
plt.ylabel('Median acousticness')

Correlation exploration

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','explicit','key','mode','year']

X, y = df[feature_names], df['popularity']

features = np.array(feature_names)

# Instantiate the visualizer
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(20,20)
visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()       

Features plot

In [ ]:
features_o = ['energy','loudness','year']

plt.rcParams['figure.figsize'] = (15, 4)

plt.subplot(1, 3, 1)
sns.distplot(df['energy'])

plt.subplot(1, 3, 2)
sns.distplot(df['loudness'])

plt.subplot(1, 3, 3)
sns.distplot(df['year'])

plt.suptitle('Checking Feature with more corre ', fontsize = 10)
plt.show()

Sve numericke atribute cemo zadrzati.

In [ ]:
df.drop(['artists', 'name', 'release_date'], axis = 1, inplace = True)
df.head()

PCA analiza


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.values
X = StandardScaler().fit_transform(X)

pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X)

In [ ]:
principalComponents

In [ ]:
PCA_dataset = pd.DataFrame(data = principalComponents, columns = ['component1', 'component2', 'component3'] )
PCA_dataset.head()

In [ ]:
principal_component1 = PCA_dataset['component1']
principal_component2 = PCA_dataset['component2']
principal_component3 = PCA_dataset['component3']

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

fig=plt.figure(figsize=(19,12.5))
ax=fig.add_subplot(2,2,1, projection="3d")
ax.scatter(principal_component1,principal_component2,principal_component3,c="blue",alpha=.3)
ax.set(xlabel='\nprincipal_component1',ylabel='\nprincipal_component2',zlabel='\nprincipal_component3')
ax.set(ylim=[0,12])

K-means algorithm with PCA selected features

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 20
kmeans = KMeans(num_clusters)
kmeans.fit(PCA_dataset)
labels = kmeans.labels_
labels

In [ ]:
df_cluster_pca = pd.concat([PCA_dataset, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster_pca.head()

In [ ]:
fig=plt.figure(figsize=(57,37.5))
ax=fig.add_subplot(2,2,1, projection="3d")
ax.scatter(df_cluster_pca["component1"][labels == 0],df_cluster_pca["component2"][labels == 0],df_cluster_pca["component3"][labels == 0], s = 100, c="red",alpha=.3, label = 'Cluster 0')
ax.scatter(df_cluster_pca["component1"][labels == 1],df_cluster_pca["component2"][labels == 1],df_cluster_pca["component3"][labels == 1], s = 100, c="green",alpha=.3, label = 'Cluster 1')
ax.scatter(df_cluster_pca["component1"][labels == 2],df_cluster_pca["component2"][labels == 2],df_cluster_pca["component3"][labels == 2], s = 100, c="blue",alpha=.3, label = 'Cluster 2')
ax.scatter(df_cluster_pca["component1"][labels == 3],df_cluster_pca["component2"][labels == 3],df_cluster_pca["component3"][labels == 3], s = 100, c="cyan",alpha=.3, label = 'Cluster 3')
ax.scatter(df_cluster_pca["component1"][labels == 4],df_cluster_pca["component2"][labels == 4],df_cluster_pca["component3"][labels == 4], s = 100, c="magenta",alpha=.3, label = 'Cluster 4')
ax.scatter(df_cluster_pca["component1"][labels == 5],df_cluster_pca["component2"][labels == 5],df_cluster_pca["component3"][labels == 5], s = 100, c="limegreen",alpha=.3, label = 'Cluster 5')
ax.scatter(df_cluster_pca["component1"][labels == 6],df_cluster_pca["component2"][labels == 6],df_cluster_pca["component3"][labels == 6], s = 100, c="hotpink",alpha=.3, label = 'Cluster 6')
ax.scatter(df_cluster_pca["component1"][labels == 7],df_cluster_pca["component2"][labels == 7],df_cluster_pca["component3"][labels == 7], s = 100, c="black",alpha=.3, label = 'Cluster 7')
ax.scatter(df_cluster_pca["component1"][labels == 8],df_cluster_pca["component2"][labels == 8],df_cluster_pca["component3"][labels == 8], s = 100, c="dimgray",alpha=.3, label = 'Cluster 8')
ax.scatter(df_cluster_pca["component1"][labels == 9],df_cluster_pca["component2"][labels == 9],df_cluster_pca["component3"][labels == 9], s = 100, c="silver",alpha=.3, label = 'Cluster 9')
ax.scatter(df_cluster_pca["component1"][labels == 10],df_cluster_pca["component2"][labels == 10],df_cluster_pca["component3"][labels == 10], s = 100, c="yellow",alpha=.3, label = 'Cluster 10')
ax.scatter(df_cluster_pca["component1"][labels == 11],df_cluster_pca["component2"][labels == 11],df_cluster_pca["component3"][labels == 11], s = 100, c="pink",alpha=.3, label = 'Cluster 11')
ax.scatter(df_cluster_pca["component1"][labels == 12],df_cluster_pca["component2"][labels == 12],df_cluster_pca["component3"][labels == 12], s = 100, c="lime",alpha=.3, label = 'Cluster 12')
ax.scatter(df_cluster_pca["component1"][labels == 13],df_cluster_pca["component2"][labels == 13],df_cluster_pca["component3"][labels == 13], s = 100, c="orange",alpha=.3, label = 'Cluster 13')
ax.scatter(df_cluster_pca["component1"][labels == 14],df_cluster_pca["component2"][labels == 14],df_cluster_pca["component3"][labels == 14], s = 100, c="darkviolet",alpha=.3, label = 'Cluster 14')
ax.scatter(df_cluster_pca["component1"][labels == 15],df_cluster_pca["component2"][labels == 15],df_cluster_pca["component3"][labels == 15], s = 100, c="deeppink",alpha=.3, label = 'Cluster 15')
ax.scatter(df_cluster_pca["component1"][labels == 16],df_cluster_pca["component2"][labels == 16],df_cluster_pca["component3"][labels == 16], s = 100, c="gold",alpha=.3, label = 'Cluster 16')
ax.scatter(df_cluster_pca["component1"][labels == 17],df_cluster_pca["component2"][labels == 17],df_cluster_pca["component3"][labels == 17], s = 100, c="powderblue",alpha=.3, label = 'Cluster 17')
ax.scatter(df_cluster_pca["component1"][labels == 18],df_cluster_pca["component2"][labels == 18],df_cluster_pca["component3"][labels == 18], s = 100, c="yellowgreen",alpha=.3, label = 'Cluster 18')
ax.scatter(df_cluster_pca["component1"][labels == 19],df_cluster_pca["component2"][labels == 19],df_cluster_pca["component3"][labels == 19], s = 100, c="tomato",alpha=.3, label = 'Cluster 19')

ax.set(xlabel='\nprincipal_component1',ylabel='\nprincipal_component2',zlabel='\nprincipal_component3')
ax.set(ylim=[0,12])


K-means algorithm with selected features

In [ ]:
df1 = df[["popularity", "tempo"]]
kmeans.fit(df1)
labels = kmeans.labels_
labels

In [ ]:
df_cluster = pd.concat([df1, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster.head()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df1["popularity"][labels == 0], df1["tempo"][labels == 0], s = 100, c = 'red', label = 'Cluster 0')
plt.scatter(df1["popularity"][labels == 1], df1["tempo"][labels == 1], s = 100, c = 'green', label = 'Cluster 1')
plt.scatter(df1["popularity"][labels == 2], df1["tempo"][labels == 2], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(df1["popularity"][labels == 3], df1["tempo"][labels == 3], s = 100, c = 'cyan', label = 'Cluster 3')
plt.scatter(df1["popularity"][labels == 4], df1["tempo"][labels == 4], s = 100, c = 'magenta', label = 'Cluster 4')
plt.scatter(df1["popularity"][labels == 5], df1["tempo"][labels == 5], s = 100, c = 'limegreen', label = 'Cluster 5')
plt.scatter(df1["popularity"][labels == 6], df1["tempo"][labels == 6], s = 100, c = 'hotpink', label = 'Cluster 6')
plt.scatter(df1["popularity"][labels == 7], df1["tempo"][labels == 7], s = 100, c = 'black', label = 'Cluster 7')
plt.scatter(df1["popularity"][labels == 8], df1["tempo"][labels == 8], s = 100, c = 'dimgray', label = 'Cluster 8')
plt.scatter(df1["popularity"][labels == 9], df1["tempo"][labels == 9], s = 100, c = 'silver', label = 'Cluster 9')
plt.scatter(df1["popularity"][labels == 10], df1["tempo"][labels == 10], s = 100, c = 'yellow', label = 'Cluster 10')
plt.scatter(df1["popularity"][labels == 11], df1["tempo"][labels == 11], s = 100, c = 'pink', label = 'Cluster 11')
plt.scatter(df1["popularity"][labels == 12], df1["tempo"][labels == 12], s = 100, c = 'lime', label = 'Cluster 12')
plt.scatter(df1["popularity"][labels == 13], df1["tempo"][labels == 13], s = 100, c = 'orange', label = 'Cluster 13')
plt.scatter(df1["popularity"][labels == 14], df1["tempo"][labels == 14], s = 100, c = 'darkviolet', label = 'Cluster 14')
plt.scatter(df1["popularity"][labels == 15], df1["tempo"][labels == 15], s = 100, c = 'deeppink', label = 'Cluster 15')
plt.scatter(df1["popularity"][labels == 16], df1["tempo"][labels == 16], s = 100, c = 'gold', label = 'Cluster 16')
plt.scatter(df1["popularity"][labels == 17], df1["tempo"][labels == 17], s = 100, c = 'powderblue', label = 'Cluster 17')
plt.scatter(df1["popularity"][labels == 18], df1["tempo"][labels == 18], s = 100, c = 'yellowgreen', label = 'Cluster 18')
plt.scatter(df1["popularity"][labels == 19], df1["tempo"][labels == 19], s = 100, c = 'tomato', label = 'Cluster 19')

plt.xlabel('Popularity')
plt.ylabel('Tempo')


In [ ]:
df2 = df[["year", "energy"]]
kmeans.fit(df2)
labels = kmeans.labels_
labels

In [ ]:
df_cluster = pd.concat([df2, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster.head()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df2["year"][labels == 0], df2["energy"][labels == 0], s = 100, c = 'red', label = 'Cluster 0')
plt.scatter(df2["year"][labels == 1], df2["energy"][labels == 1], s = 100, c = 'green', label = 'Cluster 1')
plt.scatter(df2["year"][labels == 2], df2["energy"][labels == 2], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(df2["year"][labels == 3], df2["energy"][labels == 3], s = 100, c = 'cyan', label = 'Cluster 3')
plt.scatter(df2["year"][labels == 4], df2["energy"][labels == 4], s = 100, c = 'magenta', label = 'Cluster 4')
plt.scatter(df2["year"][labels == 5], df2["energy"][labels == 5], s = 100, c = 'limegreen', label = 'Cluster 5')
plt.scatter(df2["year"][labels == 6], df2["energy"][labels == 6], s = 100, c = 'hotpink', label = 'Cluster 6')
plt.scatter(df2["year"][labels == 7], df2["energy"][labels == 7], s = 100, c = 'black', label = 'Cluster 7')
plt.scatter(df2["year"][labels == 8], df2["energy"][labels == 8], s = 100, c = 'dimgray', label = 'Cluster 8')
plt.scatter(df2["year"][labels == 9], df2["energy"][labels == 9], s = 100, c = 'silver', label = 'Cluster 9')
plt.scatter(df2["year"][labels == 10], df2["energy"][labels == 10], s = 100, c = 'yellow', label = 'Cluster 10')
plt.scatter(df2["year"][labels == 11], df2["energy"][labels == 11], s = 100, c = 'pink', label = 'Cluster 11')
plt.scatter(df2["year"][labels == 12], df2["energy"][labels == 12], s = 100, c = 'lime', label = 'Cluster 12')
plt.scatter(df2["year"][labels == 13], df2["energy"][labels == 13], s = 100, c = 'orange', label = 'Cluster 13')
plt.scatter(df2["year"][labels == 14], df2["energy"][labels == 14], s = 100, c = 'darkviolet', label = 'Cluster 14')
plt.scatter(df2["year"][labels == 15], df2["energy"][labels == 15], s = 100, c = 'deeppink', label = 'Cluster 15')
plt.scatter(df2["year"][labels == 16], df2["energy"][labels == 16], s = 100, c = 'gold', label = 'Cluster 16')
plt.scatter(df2["year"][labels == 17], df2["energy"][labels == 17], s = 100, c = 'powderblue', label = 'Cluster 17')
plt.scatter(df2["year"][labels == 18], df2["energy"][labels == 18], s = 100, c = 'yellowgreen', label = 'Cluster 18')
plt.scatter(df2["year"][labels == 19], df2["energy"][labels == 19], s = 100, c = 'tomato', label = 'Cluster 19')

plt.xlabel('Year')
plt.ylabel('Energy')

In [ ]:
df3 = df[["year", "speechiness"]]
kmeans.fit(df3)
labels = kmeans.labels_
labels

In [ ]:
df_cluster = pd.concat([df3, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster.head()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df3["year"][labels == 0], df3["speechiness"][labels == 0], s = 100, c = 'red', label = 'Cluster 0')
plt.scatter(df3["year"][labels == 1], df3["speechiness"][labels == 1], s = 100, c = 'green', label = 'Cluster 1')
plt.scatter(df3["year"][labels == 2], df3["speechiness"][labels == 2], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(df3["year"][labels == 3], df3["speechiness"][labels == 3], s = 100, c = 'cyan', label = 'Cluster 3')
plt.scatter(df3["year"][labels == 4], df3["speechiness"][labels == 4], s = 100, c = 'magenta', label = 'Cluster 4')
plt.scatter(df3["year"][labels == 5], df3["speechiness"][labels == 5], s = 100, c = 'limegreen', label = 'Cluster 5')
plt.scatter(df3["year"][labels == 6], df3["speechiness"][labels == 6], s = 100, c = 'hotpink', label = 'Cluster 6')
plt.scatter(df3["year"][labels == 7], df3["speechiness"][labels == 7], s = 100, c = 'black', label = 'Cluster 7')
plt.scatter(df3["year"][labels == 8], df3["speechiness"][labels == 8], s = 100, c = 'dimgray', label = 'Cluster 8')
plt.scatter(df3["year"][labels == 9], df3["speechiness"][labels == 9], s = 100, c = 'silver', label = 'Cluster 9')
plt.scatter(df3["year"][labels == 10], df3["speechiness"][labels == 10], s = 100, c = 'yellow', label = 'Cluster 10')
plt.scatter(df3["year"][labels == 11], df3["speechiness"][labels == 11], s = 100, c = 'pink', label = 'Cluster 11')
plt.scatter(df3["year"][labels == 12], df3["speechiness"][labels == 12], s = 100, c = 'lime', label = 'Cluster 12')
plt.scatter(df3["year"][labels == 13], df3["speechiness"][labels == 13], s = 100, c = 'orange', label = 'Cluster 13')
plt.scatter(df3["year"][labels == 14], df3["speechiness"][labels == 14], s = 100, c = 'darkviolet', label = 'Cluster 14')
plt.scatter(df3["year"][labels == 15], df3["speechiness"][labels == 15], s = 100, c = 'deeppink', label = 'Cluster 15')
plt.scatter(df3["year"][labels == 16], df3["speechiness"][labels == 16], s = 100, c = 'gold', label = 'Cluster 16')
plt.scatter(df3["year"][labels == 17], df3["speechiness"][labels == 17], s = 100, c = 'powderblue', label = 'Cluster 17')
plt.scatter(df3["year"][labels == 18], df3["speechiness"][labels == 18], s = 100, c = 'yellowgreen', label = 'Cluster 18')
plt.scatter(df3["year"][labels == 19], df3["speechiness"][labels == 19], s = 100, c = 'tomato', label = 'Cluster 19')

plt.xlabel('Year')
plt.ylabel('Speechiness')

In [ ]:
df4 = df[["instrumentalness", "speechiness"]]
kmeans.fit(df4)
labels = kmeans.labels_
labels

In [ ]:
df_cluster = pd.concat([df4, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster.head()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df4["instrumentalness"][labels == 0], df4["speechiness"][labels == 0], s = 100, c = 'red', label = 'Cluster 0')
plt.scatter(df4["instrumentalness"][labels == 1], df4["speechiness"][labels == 1], s = 100, c = 'green', label = 'Cluster 1')
plt.scatter(df4["instrumentalness"][labels == 2], df4["speechiness"][labels == 2], s = 100, c = 'blue', label = 'Cluster 2')
plt.scatter(df4["instrumentalness"][labels == 3], df4["speechiness"][labels == 3], s = 100, c = 'cyan', label = 'Cluster 3')
plt.scatter(df4["instrumentalness"][labels == 4], df4["speechiness"][labels == 4], s = 100, c = 'magenta', label = 'Cluster 4')
plt.scatter(df4["instrumentalness"][labels == 5], df4["speechiness"][labels == 5], s = 100, c = 'limegreen', label = 'Cluster 5')
plt.scatter(df4["instrumentalness"][labels == 6], df4["speechiness"][labels == 6], s = 100, c = 'hotpink', label = 'Cluster 6')
plt.scatter(df4["instrumentalness"][labels == 7], df4["speechiness"][labels == 7], s = 100, c = 'black', label = 'Cluster 7')
plt.scatter(df4["instrumentalness"][labels == 8], df4["speechiness"][labels == 8], s = 100, c = 'dimgray', label = 'Cluster 8')
plt.scatter(df4["instrumentalness"][labels == 9], df4["speechiness"][labels == 9], s = 100, c = 'silver', label = 'Cluster 9')
plt.scatter(df4["instrumentalness"][labels == 10], df4["speechiness"][labels == 10], s = 100, c = 'yellow', label = 'Cluster 10')
plt.scatter(df4["instrumentalness"][labels == 11], df4["speechiness"][labels == 11], s = 100, c = 'pink', label = 'Cluster 11')
plt.scatter(df4["instrumentalness"][labels == 12], df4["speechiness"][labels == 12], s = 100, c = 'lime', label = 'Cluster 12')
plt.scatter(df4["instrumentalness"][labels == 13], df4["speechiness"][labels == 13], s = 100, c = 'orange', label = 'Cluster 13')
plt.scatter(df4["instrumentalness"][labels == 14], df4["speechiness"][labels == 14], s = 100, c = 'darkviolet', label = 'Cluster 14')
plt.scatter(df4["instrumentalness"][labels == 15], df4["speechiness"][labels == 15], s = 100, c = 'deeppink', label = 'Cluster 15')
plt.scatter(df4["instrumentalness"][labels == 16], df4["speechiness"][labels == 16], s = 100, c = 'gold', label = 'Cluster 16')
plt.scatter(df4["instrumentalness"][labels == 17], df4["speechiness"][labels == 17], s = 100, c = 'powderblue', label = 'Cluster 17')
plt.scatter(df4["instrumentalness"][labels == 18], df4["speechiness"][labels == 18], s = 100, c = 'yellowgreen', label = 'Cluster 18')
plt.scatter(df4["instrumentalness"][labels == 19], df4["speechiness"][labels == 19], s = 100, c = 'tomato', label = 'Cluster 19')

plt.xlabel('Instrumentalness')
plt.ylabel('Speechiness')